+# Project 1: Customer Database
**This is the first of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-10-13, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
Your team has been hired by the company X as data scientists. X makes gadgets for a wide range of industrial and commercial clients.

As in-house data scientists, your teams first task, as per request from your new boss, is to optimize business operations. You have decided that a good first step would be to analyze the companys historical sales data to gain a better understanding of where profit is coming from. It may also reveal some low hanging fruit in terms of business opportunities.

To get started, you have called the IT department to get access to the customer and sales transactions database. To your horror you've been told that such a database doens't exist, and the only record of sales transactions is kept by John from finance in an Excel spreadsheet. So you've emailed John asking for a CSV dump of the spreadsheet...

In this project you need to clean the data you got from John, enrich it with further data, prepare a database for the data, and do some data analysis. The project is comprised of five parts. They are intended to be solved in the order they appear, but it is highly recommended that you read through all of them and devise an overall strategy before you start implementing anything.

## Part 1: Cleaning the data
John has emailed you the following link to the CSV dump you requested.

- [transactions.csv](https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv)

It seems as though he has been a bit sloppy when keeping the records. 

In this part you should:
- Explain what the data is
- Clean it to prepare it for inserting into a database and doing data analysis 

---
### Solution

Let us first import the packages and download the data. We only download it if we can't find it already in the directory.

In [84]:
import wget
import numpy as np
import pandas as pd
import os.path
import requests
import json
import pandas
import sqlite3
conn = sqlite3.connect('assign_db')
c = conn.cursor()
if not os.path.exists('transactions.csv'):
    link = 'https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv'
    wget.download(link)

We then take a look at the data, first by looking at the head, and then by considering the numeber of unique values and NaN's in each column.

In [3]:
data = pd.read_csv('transactions.csv')
print(data.head(n=10))

         part        company         country          city     price  \
0  54868-5165  Chatterbridge           Spain     Barcelona   784.79€   
1  60505-2867           Lajo          Greece  Thessaloniki   187.99€   
2   24385-268      Flipstorm          Greece        Athens   221.73€   
3   76117-001    Twitterbeat          France        Annecy  1075.82€   
4  44946-1046  Chatterbridge           Spain     Barcelona   412.55€   
5   16729-167  Chatterbridge           Spain     Barcelona   359.52€   
6   52125-444          Voomm          France         Paris   266.62€   
7   43419-018       Buzzbean         Germany    Düsseldorf   103.45€   
8   54092-515          Zooxo  United Kingdom        London   £704.94   
9  24286-1562           Lajo          Greece  Thessaloniki   317.65€   

                  date  
0  2016-01-02 00:01:05  
1  2016-01-02 00:05:26  
2  2016-01-02 00:18:30  
3  2016-01-02 02:32:30  
4  2016-01-02 04:51:55  
5  2016-01-02 07:20:59  
6  2016-01-02 07:40:37  
7  2016

We can see that the base of the price is not constant, shifting from EUR to GBP, which also changes the formatting. Note that the base in this case corresponds to the country, e.g United Kingdoms

In [4]:
print("Number of unique values for each column:")
print(data.nunique())
print("")
print("Number of NaN's for each column:")
print(data.isnull().sum(axis=0))
#print(data['city'].unique())

Number of unique values for each column:
part         100
company       35
country       13
city          30
price      19214
date       20552
dtype: int64

Number of NaN's for each column:
part         10
company       0
country    2171
city         33
price         1
date          0
dtype: int64
['Barcelona' 'Thessaloniki' 'Athens' 'Annecy' 'Paris' 'Düsseldorf'
 'London' 'Braga' 'Nanterre' 'Amadora\t' 'New York' 'Arnhem' 'Nice'
 'Lisbon' 'Amsterdam' 'Porto' 'Boston' 'Niihama' 'Almada' 'Aranhas'
 'Heraklion' 'Amiens' 'Patras' 'Arcueil' 'Lyon' 'Asaka' 'Champagnole'
 'Zürich' nan 'Monção' 'Vila Fria']


In [25]:
payload={'q': 'Copenhagen', 'type': 'json', 'username': 'peterbysted'}

r = requests.get('http://api.geonames.org/search', payload)

#r.json()
cities = json.loads(r.text)
print(cities)

{'status': {'message': 'user does not exist.', 'value': 10}}


In [51]:
def city_to_country(city):
    if str(city) == 'nan':
        return 'empty'
    if 'barcelona' in city.lower():
        return 'Spain'
    elif 'thessaloniki' in city.lower() or 'athens' in city.lower():
        return 'Greece'
    elif 'annecy' in city.lower() or 'paris' in city.lower() or 'nanterre' in city.lower() or 'nice' in city.lower() or \
    'nice' in city.lower() or 'amiens' in city.lower() or 'arcueil' in city.lower() or 'lyon' in city.lower() or \
    'champagnole' in city.lower():
        return 'France'
    elif 'thessaloniki' in city.lower() or 'athens' in city.lower() or 'heraklion' in city.lower() or 'patras' in city.lower():
        return 'Greece'
    elif 'braga' in city.lower() or 'amadora' in city.lower() or 'lisbon' in city.lower() or 'porto' in city.lower() \
    or 'almada' in city.lower() or 'aranhas' in city.lower() or 'monção' in city.lower() or 'vila fria' in city.lower():
        return 'Portugal'
    elif 'düsseldorf' in city.lower():
        return 'Germany'
    elif 'london' in city.lower():
        return 'United Kingdoms'
    elif 'new york' in city.lower() or 'boston' in city.lower():
        return 'USA'
    elif 'zürich' in city.lower():
        return 'Switzerland'
    elif 'amsterdam' in city.lower() or 'arnhem' in city.lower():
        return 'Netherlands'
    elif 'niihama' in city.lower() or 'asaka' in city.lower():
        return 'Japan'
for city in data['city'].unique():
    print(str(city) + ': ' + city_to_country(city))

Barcelona: Spain
Thessaloniki: Greece
Athens: Greece
Annecy: France
Paris: France
Düsseldorf: Germany
London: United Kingdoms
Braga: Portugal
Nanterre: France
Amadora	: Portugal
New York: USA
Arnhem: Netherlands
Nice: France
Lisbon: Portugal
Amsterdam: Netherlands
Porto: Portugal
Boston: USA
Niihama: Japan
Almada: Portugal
Aranhas: Portugal
Heraklion: Greece
Amiens: France
Patras: Greece
Arcueil: France
Lyon: France
Asaka: Japan
Champagnole: France
Zürich: Switzerland
nan: empty
Monção: Portugal
Vila Fria: Portugal


---
## Part 2: Enriching the data

A common task for a data scientists is to combine or enrich data from internal sources with data available from external sources. The purpose of this can be either to fix issues with the data or to make it easier to derive insights from the data.

In this part you should enrich your data with data from at least one external source. You may look to part 4 for some  inspiration as to what is required. Your solution should be automated, i.e., you can not ask the reader of your notebook to download any data manually. You should argue why and what you expect to achieve by the enrichments you are doing.

---
### Solution

In [77]:
#print(countries.head())
data_cleaned = data
for index, city in data[data['country'].isna()]['city'].iteritems():
    data_cleaned.loc[index,'country'] = city_to_country(city)
print(data_cleaned.isnull().sum(axis=0))
print(data_cleaned[data_cleaned['price'].isnull()])


part       10
company     0
country     0
city       33
price       1
date        0
dtype: int64
            part company   country     city price                 date
11015  49967-724  Roodel  Portugal  Aranhas   NaN  2017-07-11 08:00:56


In [62]:
data_cleaned[data['country'] == 'empty']['company'].unique()

array(['Ntags', 'Brainsphere', 'Thoughtmix', 'Yozio', 'Twitterbeat',
       'Zooxo', 'Zoonder', 'Wordify', 'Teklist', 'Shufflebeat',
       'Kanoodle'], dtype=object)

We still have 33 cities which are undecided, maybe we can see how many cities each company has used before, to see if we can fill them out

In [75]:
for company in data_cleaned[data['country'] == 'empty']['company'].unique():
   print(str(company) + ": " + str(data[data['company'] == company]['city'].unique()))

Ntags: ['Lisbon' nan]
Brainsphere: ['Braga' nan 'Monção']
Thoughtmix: ['Amadora\t' 'Vila Fria' nan]
Yozio: ['Patras' nan]
Twitterbeat: ['Annecy' nan]
Zooxo: ['London' nan]
Zoonder: ['Boston' nan]
Wordify: ['New York' nan]
Teklist: ['Arnhem' nan]
Shufflebeat: ['Porto' nan]
Kanoodle: ['Niihama' nan]


That means we should be able to fill out the remaining cities for most of the companies.

 ---
## Part 3: Creating a database
Storing data in a relational database has the advantages that it is persistent, fast to query, and it will be easier access for other employees at Weyland-Yutani.

In this part you should:
- Create a database and table(s) for the data
- Insert data into the tables

You may use SQLite locally to do this. You should argue why you choose to store your data the way you do. 

---
### Solution

In [86]:
#data_cleaned.to_sql(name = 'customer_db',con=conn)
c.execute('SELECT * FROM customer_db')
c.fetchall()

[(0,
  '54868-5165',
  'Chatterbridge',
  'Spain',
  'Barcelona',
  '784.79€',
  '2016-01-02 00:01:05'),
 (1,
  '60505-2867',
  'Lajo',
  'Greece',
  'Thessaloniki',
  '187.99€',
  '2016-01-02 00:05:26'),
 (2,
  '24385-268',
  'Flipstorm',
  'Greece',
  'Athens',
  '221.73€',
  '2016-01-02 00:18:30'),
 (3,
  '76117-001',
  'Twitterbeat',
  'France',
  'Annecy',
  '1075.82€',
  '2016-01-02 02:32:30'),
 (4,
  '44946-1046',
  'Chatterbridge',
  'Spain',
  'Barcelona',
  '412.55€',
  '2016-01-02 04:51:55'),
 (5,
  '16729-167',
  'Chatterbridge',
  'Spain',
  'Barcelona',
  '359.52€',
  '2016-01-02 07:20:59'),
 (6,
  '52125-444',
  'Voomm',
  'France',
  'Paris',
  '266.62€',
  '2016-01-02 07:40:37'),
 (7,
  '43419-018',
  'Buzzbean',
  'Germany',
  'Düsseldorf',
  '103.45€',
  '2016-01-02 08:57:57'),
 (8,
  '54092-515',
  'Zooxo',
  'United Kingdom',
  'London',
  '£704.94',
  '2016-01-02 09:09:01'),
 (9,
  '24286-1562',
  'Lajo',
  'Greece',
  'Thessaloniki',
  '317.65€',
  '2016-01-02 11

---
## Part 4: Analyzing the data
You are now ready to analyze the data. Your goal is to gain some actionable business insights to present to your boss. 

In this part, you should ask some questions and try to answer them based on the data. You should write SQL queries to retrieve the data. For each question, you should state why it is relevant and what you expect to find.

To get you started, you should prepare answers to the following questions. You should add more questions.
#### Who are the most profitable clients?
Knowing which clients that generate the most revenue for the company will assist your boss in distributing customer service ressources.

#### Are there any clients for which profit is declining?
Declining profit from a specific client may indicate that the client is disatisfied with the product. Gaining a new client is often much more work than retaining one. Early warnings about declining profit may help your boss fighting customer churn.


Remember, you are taking this to your new boss, so think about how you present the data.

---
### Solution

---
## Part 5: Performance
Your boss is very impressed with what you have achieved in less than two weeks, and he would like to take your idea of storing the customer and sales data in a relational database to production. However, John is concerned that the solution will not scale. His experience is telling him that you will see many occurrences of the following queries.

- Show all sales to company X between time $t_1$ and time $t_2$
- Show the latest X sales in the database
- Show total sales per company per day

Show that Johns concern is not justified.

---
### Solution

---